In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_group(pappas_only=False)

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])

In [4]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [7]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

for i in range(10):
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, X_STSIMM, label_material, label_visiprog, \
                     test_size=0.2, stratify=label_material)

    res = leave_one_sample_out(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test)
    accuracy_STSIMM.append(res['accuracy'])

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(np.nan)

    model_vsp_original = LFDA()
    try:
        # remove the unlabeled data
        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
        
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        accuracy_vsp_original.append(np.nan)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)

INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
I

In [8]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])
df.loc['average'] = df.mean()
display(df)

,STSIMM,material,visiprog_original,visiprog
0,0.849380,0.938990,0.861773,0.938990
1,0.862726,0.938990,0.846520,0.932316
2,0.846520,0.938990,0.846520,0.927550
3,0.867493,0.940896,0.871306,0.939943
4,0.872259,0.946616,0.855100,0.945663
5,0.838894,0.943756,0.829361,0.941849
6,0.855100,0.936130,0.853194,0.930410
7,0.849380,0.945663,0.855100,0.918017
8,0.850334,0.946616,0.847474,0.943756
9,0.862726,0.940896,0.874166,0.932316


In [ ]:
print(Y_mat_test)